# Assignment

In this assignment we will create a model for segmentation of tumor from abdominal CT images using custom loss function modifications to increase prediction sensitivity.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 89 kB 4.0 MB/s 
     |████████████████████████████████| 596 kB 31.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [2]:
import numpy as np, pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, models, losses, layers, optimizers
from jarvis.train import datasets
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of kidney tumor CT exams derived from the Kidney Tumor Segmentation Challenge (KiTS). More information about he KiTS Challenge can be found here: https://kits21.kits-challenge.org/. In this exercise, we will use this dataset to derive a model for slice-by-slice kidney segmentation. The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/ct_kits`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [3]:
# --- Download dataset
datasets.download(name='ct/kits')

[ 2022-05-21 18:57:53 ] [====================] 100.000% : Extracting archive (0000818 / 0000818) 

{'code': '/data/raw/ct_kits', 'data': '/data/raw/ct_kits'}

# Training

### Stratified Sampling

Use the following code block to define a custom configuration dictionary to increase the sampling distribution of tumor (`lbl-crp-02`) up to 30%:

In [4]:
# --- Configs dict to implement stratified sampling
configs = {
    'batch': {'size': 16},
    'sampling': {
        'lbl-crp-00': 0.4,
        'lbl-crp-01': 0.3,
        'lbl-crp-02': 0.3}}

# --- Prepare generators
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)

In the assignment, feel free to experiment with different stratified sampling distributions. 

### Define the backbone model

Use the following cell block to define your backbone for the semantic segmentation task:

In [5]:
# lambda and kwargs

kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'}

conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

concat = lambda a, b : layers.Concatenate()([a, b])

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2))))

In [6]:
# --- Define input
x = Input(shape=(None, 96, 96, 1), dtype='float32')

# --- Define model
l1 = conv1(8, x)
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))

l6  = tran2(48, l5)
l7  = tran2(32, conv1(48, concat(l4, l6)))
l8  = tran2(16, conv1(32, concat(l3, l7)))
l9  = tran2(8,  conv1(16, concat(l2, l8)))
l10 = conv1(8,  l9)
# --- Define logits
logits = layers.Conv3D(filters=2, **kwargs)(l10)


# --- Create model
backbone = Model(inputs=x, outputs=logits) 

### Define training model

Use the following cell block to start building your training model using the backbone network.

In [7]:
# --- Define inputs
inputs = {
    'dat': Input(shape=(None, 96, 96, 1), dtype='float32'),
    'lbl': Input(shape=(None, 96, 96, 1), dtype='uint8')}
# --- Define model
logits = backbone(inputs['dat'])

### Custom loss function

In order to create a high sensitivity classifier for tumor segmentation, a combined weighted and masked loss strategy should be implemented. More specifically, the following weighting tensor should be create:

* class 0 (background; non-kidney): set `wgt` to 0
* class 1 (background; kidney): set `wgt` to 1
* class 2 (foreground; tumor): set `wgt` to positive value

In addition, recall that you will need to convert the three-class ground-truth label into a binarized target label (tumor or no tumor).

In [8]:
#functions

def create_weights(lbl, pos_weight=5.0):

    pass

In [9]:
# --- Create weights
wgt = create_weights(inputs['lbl'])
# --- Create y_true (binarized ground-truth)
y_true = tf.cast(inputs['lbl'] == 2, dtype='uint8')
# --- Create loss
sce = losses.SparseCategoricalCrossentropy(from_logits=True)(
    y_true=y_true,
    y_pred=logits,
    sample_weight=wgt)

### Custom metrics

The goal of weighted and/or masked loss functions in this example is maximize the sensitivity for tumor prediction. Thus, in addition to a standard Dice score metric, we will additionally use foreground sensitivity to track overall model performance. Recall that to adjust the metrics to account for a custom weighted loss function, one must simply ignore predictions from masked regions (e.g., the model is required to predict accurate results in these regions). 

In [10]:
def calculate_dsc(y_true, y_pred, weights=None, c=1):
    """
    Method to calculate the Dice score coefficient for given class
    
    :params
    
      y_true : ground-truth label
      y_pred : predicted logits scores
           c : class to calculate DSC on
    
    """  
    true = y_true[..., 0] == c
    pred = tf.math.argmax(y_pred, axis=-1) == c 
    
    if weights is not None:
        true = true & (weights[..., 0] != 0)
        pred = pred & (weights[..., 0] != 0)

    A = tf.math.count_nonzero(true & pred) * 2
    B = tf.math.count_nonzero(true) + tf.math.count_nonzero(pred)
    
    return tf.math.divide_no_nan(
        tf.cast(A, tf.float32), 
        tf.cast(B, tf.float32))

In [11]:
def calculate_sen(y_true, y_pred, weights=None, c=1, **kwargs):
    """
    Method to implement sensitivity (recall) on raw cross-entropy logits

    """
    true = y_true[..., 0] == c
    pred = tf.math.argmax(y_pred, axis=-1) == c 
    
    if weights is not None:
        true = true & (weights[..., 0] != 0)
        pred = pred & (weights[..., 0] != 0)
        
    tp = true & pred

    num = tf.math.count_nonzero(tp) 
    den = tf.math.count_nonzero(y_true)

    num = tf.cast(num, tf.float32)
    den = tf.cast(den, tf.float32)

    return tf.math.divide_no_nan(num, den)

In [12]:
# --- Create metrics
dsc = calculate_dsc(y_true, logits, wgt)
sen = calculate_sen(y_true, logits, wgt)

Now, we are ready to create the `training` model and add the corresponding loss and accuracy tensors. 

In [13]:
# --- Create model
training = Model(inputs=inputs, outputs={'logits': logits, 'dsc': dsc, 'sen': sen})
# --- Add loss
training.add_loss(sce)

# --- Add metrics
training.add_metric(dsc, name='dsc')
training.add_metric(sen, name='sen')

### Compile the model

Use the following cell block to compile your model with an appropriate optimizer. 

In [14]:
optimizer = optimizers.Adam(learning_rate=2e-4)

training.compile(optimizer=optimizer)

### In-memory data

To speed up training, consider loading all your model data into RAM memory:

In [15]:
# --- Load data into memory for faster training
client.load_data_in_memory()

[ 2022-05-21 19:02:01 ] [====================] 100.000% : Iterating | 000402    

### Train the model

Use the following cell block to train your model.

In [17]:
training.fit(
    x=gen_train, 
    steps_per_epoch=100, 
    epochs=20,
    validation_data=gen_valid,
    validation_steps=100,
    validation_freq=5,
    use_multiprocessing=True)

Epoch 1/20
100/100 [==============================] - 20s 23ms/step - loss: 0.6220 - dsc: 0.0406 - sen: 0.3127
Epoch 2/20
100/100 [==============================] - 2s 25ms/step - loss: 0.2796 - dsc: 0.0102 - sen: 0.0119
Epoch 3/20
100/100 [==============================] - 3s 25ms/step - loss: 0.1567 - dsc: 0.0490 - sen: 0.0262
Epoch 4/20
100/100 [==============================] - 2s 25ms/step - loss: 0.1043 - dsc: 0.0862 - sen: 0.0466
Epoch 5/20
100/100 [==============================] - 5s 50ms/step - loss: 0.0785 - dsc: 0.1513 - sen: 0.0864 - val_loss: 0.0746 - val_dsc: 0.0498 - val_sen: 0.0259
Epoch 6/20
100/100 [==============================] - 2s 24ms/step - loss: 0.0652 - dsc: 0.2816 - sen: 0.1804
Epoch 7/20
100/100 [==============================] - 2s 24ms/step - loss: 0.0560 - dsc: 0.5457 - sen: 0.4405
Epoch 8/20
100/100 [==============================] - 3s 25ms/step - loss: 0.0504 - dsc: 0.6328 - sen: 0.5645
Epoch 9/20
100/100 [==============================] - 3s 25ms/st

# Evaluation

Based on the tutorial discussion, use the following cells to calculate model performance. The following metrics should be calculated:

* pixel-wise sensitivity (mean, median, 25th percentile, 75th percentile)
* Dice score coefficient (mean, median, 25th percentile, 75th percentile)

### Performance

The following minimum performance metrics must be met for full credit:

* median pixel-wise sensitivity: >0.70
* median Dice score coefficient: >0.70

In [18]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort sensitivity and Dice score statistics. There is no need to submit training performance accuracy.

In [19]:
x, y = next(test_valid)
outputs = training.predict(x)

test_train, test_valid = client.create_generators(test=True)

dice = []
sens = []

for x, y in test_valid:
    
    if (x['lbl'] == 2).any():
    
        outputs = training.predict(x)

        dice.append(outputs['dsc'])

        sens.append(outputs['sen'])

dice = np.array(dice)
sens = np.array(sens)

df = pd.DataFrame()
df['dice'] = dice
df['sens'] = sens

[ 2022-05-21 19:04:54 ] [====================] 100.000% : Iterating | 000402    

In [21]:
df.to_csv('./wjhan_results.csv')

# Submission

Use the following line to save your model for submission:

In [22]:
# --- Serialize a model
backbone.save('./wjhan_model.hdf5')

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.